LNB Chassis thermal analysis
====
Last update: 30 Aug 2018  
  
The frequency converters for the phased array antenna reside several 3U chassis with forced air cooling.  The total power dissipated in each chassis is high, so this analysis will estimate the air exit temperature and the surface temperature of the PCB.  This thermal analysis is approximate because of some simpliflying assumptions, however the accuracy should be adaquate for an initial evaluation.  
  
Reference:  
(1) Heat Transfer, A Practical Approach, 2nd Edition, by Yunus Cengel  

In [2]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

## Properties of air at 1 atmosphere
Select data taken from Table A-15, page 874.

In [3]:
temp = np.array([-100,-50,-40,-30,-20,-10,0,5,10,15,20,25,30,35,40,45,50,60,70,80,90,100,120],dtype=float)
thermal_conduc = np.array([.01582,.01979,.02057,.02134,.02211,.02288,.02364,.02401,.02439,.02476,.02514,
    .02551,.02588,.02625,.02662,.02699,.02735,.02808,.02881,.02953,.03024,.03095,.03235])
kinem_visc = np.array([.5837,.9319,1.008,1.087,1.169,1.252,1.338,1.382,1.426,1.470,1.516,1.562,1.608,
    1.655,1.702,1.750,1.798,1.896,1.995,2.097,2.201,2.306,2.522])*1e-5
Prandle_num = np.array([.7263,.7440,.7436,.7425,.7408,.7387,.7362,.7350,.7336,.7323,.7309,.7296,.7282,
    .7268,.7255,.7241,.7228,.7202,.7177,.7154,.7132,.7111,.7073])
air_density = np.array([2.038,1.582,1.514,1.451,1.394,1.341,1.292,1.269,1.246,1.225,1.204,1.184,1.164,
    1.145,1.127,1.109,1.092,1.059,1.028,.9994,.9718,.9458,.8977])
spec_heat = np.array([966,999,1002,1004,1005,1006,1006,1006,1006,1007,1007,1007,1007,1007,1007,1007,
    1007,1007,1007,1008,1008,1009,1011],dtype=float)

# L-Band
1 chassis, 7U size = 12.5 inches   
4 combiner CCA per chassis, W=14.5 inches L=13.3  
1 cal distro  
70 mixer CCA  
673 Watts total  
  
## Calculate air exit temperature
The heat dissipated by the electronics must be absorbed by the air, the required mass flow rate of air needed to absorb the heat can be determined from:  
$Q = m*Cp(T_{out}-T_{in})$  
  
The fans were not specified, for now assume 100 CFM and 17 W per fan and a total of 3 fans per chassis.  Assume that power used by the fan is transfered to the air.  
Typically 5 inch fans have a CFM range between 80 and 144.  

In [11]:
# Fan specification
Pwr_fan = 17 # Watts used by fan
num_fans = 6
CFM_per_fan = 100 # ft^3/min (CFM)
V_dot = num_fans*CFM_per_fan*0.0283168/60 # convert to m^3/s
print('volume flow rate = {:.3f} m^3/s'.format(V_dot))

T_in = 20  # nominal room temperature, C
Cp = np.interp(T_in,temp,spec_heat) # specific heat of air, see table A-15
rho = np.interp(T_in,temp,air_density) # specific heat of air, see table A-15
print('air density = {:.3f} kg/m^3'.format(rho))

m_dot = rho*V_dot # mass flow rate
print('mass flow rate = {:.3f} kg/s'.format(m_dot))

Q_dot_fan = Pwr_fan
Q_chassis = 637  # power provided in WG#1 slides
Q_tot = Q_chassis + Q_dot_fan*num_fans  # heat generated in the chassis, W
T_exit = fsolve((lambda T_exit : m_dot*Cp*(T_exit-T_in) - Q_tot),100.0) # solve for exit temperature
T_exit = float(T_exit)
print('exit air temperature = {:.3f} C'.format(T_exit))

volume flow rate = 0.283 m^3/s
air density = 1.204 kg/m^3
mass flow rate = 0.341 kg/s
exit air temperature = 22.153 C


## Calculate the surface temperature of the PCB
The dimensions of the PCBs are estimated.  The rack mount chassis is assumed to be 19 wide and 20 inches deep. The internal PCB are assumed to be 18 by 19 inches.  The boards are mounted horizontally in the chassis.  

In [14]:
# estimate the size of the PCB
L = 15*0.0254  # length of PCB, convert to m
W = 17.5*0.0254  # wide of PCB, convert to m
gap = (12.5/5)*0.0254 # space between boards
As = W*L*2 # heat transfer surface area, both sides

# cross section area and hydraulic diameter
Ac = gap*W  #  only look at one channel
perimeter = 2*gap+2*W
Dh = 4*Ac/perimeter  # hydraulic diameter, see page 442 in text book
print('hydraulic diameter  = {:.3f} m'.format(Dh))

# velocity of air
num_pcb = 5  # number of PCB forming the channel
Ac = gap*W*num_pcb  # area of cross section of X channels
vel_air = V_dot/Ac # velocity of air
print('velocity of air  = {:.3f} m/s'.format(vel_air))

# properties of air
v = np.interp(T_in,temp,kinem_visc)  # kinematic viscosity air at 1 atm, see table A-15
Pr = np.interp(T_in,temp,Prandle_num)  # Prandtl number, see table A-15
k = np.interp(T_in,temp,thermal_conduc) # thermal conductivity of air, see table A-15

# calculate Reynolds and Nusselt number, length of hydrodynamic entrance region is ignored
Re = vel_air*Dh/v # Renolds number for air in a rectangular duct
print('Reynolds number = {:,.1f}'.format(Re))
if Re < 2300:
    print('flow is laminar')
    # Nusselt number from Table 15-3, with aspect greater than 8
    Nu = 8.24 # see table 8-1, page 437
if 2300 <= Re and Re < 10000:
    print('flow is transitional')
    # interpolate a Nu value between laminar and turbulent values
    Nu = ((0.023*Re**0.8*Pr**0.4)-8.24)*Re/(2300+10000)+8.24
if Re >= 10000:
    print('flow is turbulent')
    Nu = 0.023*Re**0.8*Pr**0.4  # see equation 8-68, page 441, Dittus-Boelter equation

print('Nusselt number = {:.3f}'.format(Nu))

h = k*Nu/Dh
print('heat transfer coefficient = {:.3f} W/m^2.C'.format(h))

# disreguarding the entrance effects, the temperature difference between the surface of the PCB and the air along the channel.
# Q = h*As*(T_delta)
Q_pcb = Q_chassis/num_pcb  # calculate power dissipated by each PCB
T_delta = Q_pcb/(h*As) # calculate the temperature difference between the surface of the PCB and the air
print('delta temperature  = {:.1f} C'.format(T_delta))
print('PCB surface temperature  = {:.1f} C'.format(T_exit+T_delta)) # air at the exit is assumed to be the highest

hydraulic diameter  = 0.111 m
velocity of air  = 2.006 m/s
Reynolds number = 14,707.6
flow is turbulent
Nusselt number = 43.783
heat transfer coefficient = 9.905 W/m^2.C
delta temperature  = 38.0 C
PCB surface temperature  = 60.1 C


# S-Band
4 chassis  
6U size = 10.5 inches
4 combiner CCA per chassis  
16 mixer CCA  
estimated size of PCB: W = 17.5 inches, L = 15 inches
2219 Watts total  
## Calculate air exit temperature

In [17]:
# Fan specification
Pwr_fan = 17 # Watts used by fan
num_fans = 6
CFM_per_fan = 100 # ft^3/min (CFM)
V_dot = num_fans*CFM_per_fan*0.0283168/60 # convert to m^3/s
print('volume flow rate = {:.3f} m^3/s'.format(V_dot))

T_in = 20  # nominal room temperature, C
Cp = np.interp(T_in,temp,spec_heat) # specific heat of air, see table A-15
rho = np.interp(T_in,temp,air_density) # specific heat of air, see table A-15
print('air density = {:.3f} kg/m^3'.format(rho))

m_dot = rho*V_dot # mass flow rate
print('mass flow rate = {:.3f} kg/s'.format(m_dot))

Q_dot_fan = Pwr_fan
num_chassis = 4
Q_chassis = 2219/num_chassis
Q_tot = Q_chassis + Q_dot_fan*num_fans  # heat generated in the chassis, W
T_exit = fsolve((lambda T_exit : m_dot*Cp*(T_exit-T_in) - Q_tot),100.0) # solve for exit temperature
T_exit = float(T_exit)
print('exit air temperature = {:.3f} C'.format(T_exit))

volume flow rate = 0.283 m^3/s
air density = 1.204 kg/m^3
mass flow rate = 0.341 kg/s
exit air temperature = 21.913 C


## Calculate the surface temperature of the PCB

In [18]:
L = 15*0.0254  # length of PCB, convert to m
W = 17.5*0.0254  # wide of PCB, convert to m
gap = (10.5/4)*0.0254 # space between boards
As = W*L*2 # heat transfer surface area, both sides

# cross section area and hydraulic diameter
Ac = gap*W  #  only look at one channel
perimeter = 2*gap+2*W
Dh = 4*Ac/perimeter  # hydraulic diameter
print('hydraulic diameter  = {:.3f} m'.format(Dh))

# velocity of air
num_pcb = 4
Ac = gap*W*num_pcb  # area of cross section of X channels
vel_air = V_dot/Ac # velocity of air
print('velocity of air  = {:.3f} m/s'.format(vel_air))

# properties of air at the entry
v = np.interp(T_in,temp,kinem_visc)  # kinematic viscosity air at 1 atm, see table A-15
Pr = np.interp(T_in,temp,Prandle_num)  # Prandtl number, see table A-15
k = np.interp(T_in,temp,thermal_conduc) # thermal conductivity of air, see table A-15

# Reynolds number
Re = vel_air*Dh/v
print('Reynolds number = {:,.1f}'.format(Re))
if Re < 2300:
    print('flow is laminar')
    # Nusselt number from Table 15-3, with aspect greater than 8
    Nu = 8.24 #
if 2300 <= Re and Re < 10000:
    print('flow is transitional')
    # interpolate a Nu value between laminar and turbulent values
    Nu = ((0.023*Re**0.8*Pr**0.4)-8.24)*Re/(2300+10000)+8.24
if Re >= 10000:
    print('flow is turbulent')
    Nu = 0.023*Re**0.8*Pr**0.4  # see equation 8-68 in text, Colburn equation

print('Nusselt number = {:.3f}'.format(Nu))

h = k*Nu/Dh
print('heat transfer coefficient = {:.1f} W/m^2.C'.format(h))

# disreguarding the entrance effects, the temperature difference between the surface of the PCB and the air along the channel.
# Q = h*As*(T_delta)
Q_pcb = Q_chassis/num_pcb
T_delta = Q_pcb/(h*As)
print('delta temperature  = {:.1f} C'.format(T_delta))
print('PCB surface temperature  = {:.1f} C'.format(T_exit+T_delta))

hydraulic diameter  = 0.116 m
velocity of air  = 2.389 m/s
Reynolds number = 18,270.3
flow is turbulent
Nusselt number = 52.080
heat transfer coefficient = 11.3 W/m^2.C
delta temperature  = 36.3 C
PCB surface temperature  = 58.2 C


## C-Band
6 chassis, 7U size = 12.5 inches   
4 combiner CCA per chassis, W=14.5 inches L=13.3  
1 cal distro  
16 mixer CCA  
9312 Watts total  
## Calculate air exit temperature

In [22]:
# Fan specification
Pwr_fan = 17 # Watts used by fan
num_fans = 6
CFM_per_fan = 100 # ft^3/min (CFM)
V_dot = num_fans*CFM_per_fan*0.0283168/60 # convert to m^3/s
print('volume flow rate = {:.3f} m^3/s'.format(V_dot))

T_in = 20  # nominal room temperature, C
Cp = np.interp(T_in,temp,spec_heat) # specific heat of air, see table A-15
rho = np.interp(T_in,temp,air_density) # specific heat of air, see table A-15
print('air density = {:.3f} kg/m^3'.format(rho))

m_dot = rho*V_dot # mass flow rate
print('mass flow rate = {:.3f} kg/s'.format(m_dot))

Q_dot_fan = Pwr_fan
num_chassis = 6
Q_chassis = 9312/num_chassis
Q_tot = Q_chassis + Q_dot_fan*num_fans  # heat generated in the chassis, W
T_exit = fsolve((lambda T_exit : m_dot*Cp*(T_exit-T_in) - Q_tot),100.0) # solve for exit temperature
T_exit = float(T_exit)
print('exit air temperature = {:.3f} C'.format(T_exit))

volume flow rate = 0.283 m^3/s
air density = 1.204 kg/m^3
mass flow rate = 0.341 kg/s
exit air temperature = 24.818 C


## Calculate the surface temperature of the PCB

In [23]:
L = 15*0.0254  # length of PCB, convert to m
W = 17.5*0.0254  # wide of PCB, convert to m
gap = (12.5/5)*0.0254 # space between boards
As = W*L*2 # heat transfer surface area, both sides

# cross section area and hydraulic diameter
Ac = gap*W  #  only look at one channel
perimeter = 2*gap+2*W
Dh = 4*Ac/perimeter  # hydraulic diameter
print('hydraulic diameter  = {:.3f} m'.format(Dh))

# velocity of air
num_pcb = 5
Ac = gap*W*num_pcb  # area of cross section of X channels
vel_air = V_dot/Ac # velocity of air
print('velocity of air  = {:.3f} m/s'.format(vel_air))

# properties of air
v = np.interp(T_in,temp,kinem_visc)  # kinematic viscosity air at 1 atm, see table A-15
Pr = np.interp(T_in,temp,Prandle_num)  # Prandtl number, see table A-15
k = np.interp(T_in,temp,thermal_conduc) # thermal conductivity of air, see table A-15

# Reynolds number
Re = vel_air*Dh/v
print('Reynolds number = {:,.1f}'.format(Re))
if Re < 2300:
    print('flow is laminar')
    # Nusselt number from Table 15-3, with aspect greater than 8
    Nu = 8.24 #
if 2300 <= Re and Re < 10000:
    print('flow is transitional')
    # interpolate a Nu value between laminar and turbulent values
    Nu = ((0.023*Re**0.8*Pr**0.4)-8.24)*Re/(2300+10000)+8.24
if Re >= 10000:
    print('flow is turbulent')
    Nu = 0.023*Re**0.8*Pr**0.4  # see equation 8-68 in text, Colburn equation

print('Nusselt number = {:.3f}'.format(Nu))

h = k*Nu/Dh
print('heat transfer coefficient = {:.1f} W/m^2.C'.format(h))

# disreguarding the entrance effects, the temperature difference between the surface of the PCB and the air along the channel.
# Q = h*As*(T_delta)
Q_pcb = Q_chassis/num_pcb
T_delta = Q_pcb/(h*As)
print('delta temperature  = {:.1f} C'.format(T_delta))
print('PCB surface temperature  = {:.1f} C'.format(T_exit+T_delta))

hydraulic diameter  = 0.111 m
velocity of air  = 2.006 m/s
Reynolds number = 14,707.6
flow is turbulent
Nusselt number = 43.783
heat transfer coefficient = 9.9 W/m^2.C
delta temperature  = 92.5 C
PCB surface temperature  = 117.3 C
